<a href="https://colab.research.google.com/github/atjay2002/ANIMAL-CLASSIFICATION-IN-DEEP-LEARNING/blob/main/IMAGECLASSIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

Set path to dataset

In [ ]:
train_dir = '/content/drive/MyDrive/animal_dataset/train'
validation_dir = '/content/drive/MyDrive/animal_dataset/test'

Setting the parameters

In [ ]:
num_classes = 43
image_size = (299, 299)
batch_size = 32
learning_rate = 0.001
epochs = 5

Preprocessing and augument the training data

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

Preprocess the validation data

In [ ]:
valid_datagen = ImageDataGenerator(rescale=1./255)

Load the inceptionv3 model

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

87910968/87910968 [==============================] - 0s 0us/step


Adding the top layer for the model

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

Combine base model with top layer

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

Freeze the layers in base model

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

Compile the model

In [ ]:
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

Generate the training and validation data from directories

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 2120 images belonging to 43 classes.
Found 430 images belonging to 43 classes.


Train the model

In [ ]:
historyhistory = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs,
    validation_data=valid_generator,
    validation_steps=valid_generator.n // batch_size
)

Epoch 1/5
66/66 [==============================] - 1201s 18s/step - loss: 1.0867 - accuracy: 0.7371 - val_loss: 0.5278 - val_accuracy: 0.8726
Epoch 2/5
66/66 [==============================] - 745s 11s/step - loss: 0.2826 - accuracy: 0.9018 - val_loss: 0.5284 - val_accuracy: 0.8774
Epoch 3/5
66/66 [==============================] - 721s 11s/step - loss: 0.2063 - accuracy: 0.9363 - val_loss: 0.4963 - val_accuracy: 0.8942
Epoch 4/5
66/66 [==============================] - 721s 11s/step - loss: 0.1573 - accuracy: 0.9502 - val_loss: 0.4841 - val_accuracy: 0.8990
Epoch 5/5
66/66 [==============================] - 724s 11s/step - loss: 0.1371 - accuracy: 0.9531 - val_loss: 0.4875 - val_accuracy: 0.9062


Testing the model

In [ ]:
import PIL.Image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


model = tf.keras.models.load_model('/content/drive/MyDrive/inceptionv3_model.h5')


class_labels = ['bear', 'bison', 'boar', 'cat', 'chimpanzee', 'cow', 'coyote', 'deer', 'donkey',
                'elephant', 'flamingo', 'fox', 'goat', 'goose', 'gorilla', 'grasshopper', 'hamster',
                'hare', 'hedgehog', 'hippopotamus', 'hornbill', 'horse', 'hyena', 'kangaroo', 'koala',
                'leopard', 'lion', 'lobster', 'moth', 'okapi', 'orangutan', 'ox', 'panda', 'pelecaniformes',
                'pig', 'porcupine', 'possum', 'reindeer', 'rhinoceros', 'sheep', 'tiger', 'wolf', 'zebra']


def preprocess_image(image_path):
    img = PIL.Image.open(image_path)
    img = img.resize((299, 299))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return img


def predict_animal(image_path):
    processed_image = preprocess_image(image_path)
    predictions = model.predict(processed_image)
    predicted_label = class_labels[np.argmax(predictions)]
    confidence = np.max(predictions)
    return predicted_label, confidence


from google.colab import files
uploaded = files.upload()

uploaded_file_path = list(uploaded.keys())[0]


prediction, confidence = predict_animal(uploaded_file_path)


img = PIL.Image.open(uploaded_file_path)
plt.imshow(img)
plt.axis('off')
plt.title(f'Prediction: {prediction}\nConfidence: {confidence:.2%}')
plt.show()